In [1]:
from elasticsearch import Elasticsearch
import sqlite3
import json

## 1. Initialization

In [2]:
es = Elasticsearch(hosts='localhost')

In [4]:
mappings = {
    "mappings": {
        "properties": {
            "id": {
                "type": "integer"
            },
            "url": {
                "type": "text"
            },
            "description": {
                "type": "text",
                "analyzer": "english",
                "fielddata": True
            },
            "article_id": {
                "type": "integer"
            },
            "title": {
                "type": "text",
                "analyzer": "english",
                "fielddata": True
            },
            "author": {
                "type": "text",
                "analyzer": "english",
                "fielddata": True
            },
            "date": {
                "type": "text",
                "fielddata": True
            }
        }
    }
}
es.indices.create(index='images', body=mappings)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'images'}

## 2. Load data

In [5]:
db = sqlite3.connect('../data/MSN_technology.db')
cursor = db.cursor()
sql_text = 'SELECT * FROM IMAGES'
info_all = list(cursor.execute(sql_text))

In [6]:
for (_id, url, description, article_id, title, author, date) in info_all:
    print(_id, description)

    es.index(index='images', body=json.dumps({
        'id': _id,
        'url': url,
        'description': description,
        'article_id': article_id,
        'title': title,
        'author': author,
        'date': date
    }))

5 a group of people standing next to a palm tree: Google installed an actual ride in its booth back in 2019. James Martin/CNET
16036 a group of people sitting at a table: Nothing will top Google installing an actually ride in its booth back in 2019. 
16037 a group of people standing around a table: Google "trapped" its guests, requiring them to use Google Assistant to make their way out. James Martin/CNET
16038 a group of people performing on a counter: Google went all out with its setup in 2019. James Martin/CNET
16039 a group of people standing in a room: Google's 2018 booth was part fun house, part pitch for Assistant. Sarah Tew/CNET
16040 a woman standing in front of a window: UK Coronavirus Mutation
16041 diagram: These best-selling KN95 masks used to cost $45 at Amazon
16042 a desk with a computer mouse on a table: MailOnline logo
16043 a close up of a mouse: These wireless earbuds have proved to be a popular choice among Amazon shoppers, with reviewers praising the audio quality

## 3. Cleanups

In [7]:
db.close()
es.close()